In [17]:
import numpy as np
import pandas as pd
import PIL

In [18]:
df_images = pd.read_csv('images.csv')
df_products = pd.read_csv('clean_products.csv', lineterminator='\n')

In [19]:
df_all = pd.merge(df_products[['id', 'category_id']], df_images[['product_id', 'id', 'bucket_link']], left_on='id', right_on='product_id').copy()

In [20]:
df_all['img_file_name'] = df_all['bucket_link'].str.split('/').str[-1]
df_all['img_array'] = np.nan

In [ ]:
df_all.to_csv('image_cat.csv')

In [5]:
from PIL import Image
from numpy import asarray

df_all['img_array'] = df_all['img_array'].astype(object)

for idx, row in df_all.iterrows():
    img_file = f"./images_clean/{row['img_file_name']}"
    try:
        img = Image.open(img_file)
        
        # convert to array and flatten to 1d
        img_array = asarray(img)
        
        df_all.at[idx, 'img_array'] = img_array

    except FileNotFoundError:
        print(f'No file for {img_file}')

df_all.to_pickle('prod_imgs.pkl')

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
import pandas as pd


In [ ]:
df_all = pd.read_pickle('prod_imgs.pkl')

In [10]:
import numpy as np
from sklearn.feature_extraction import image

from sklearn.feature_extraction.image import PatchExtractor
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV

one_image = df_all.img_array[0]
patches = image.extract_patches_2d(one_image, (2, 2), max_patches=2, random_state=0)
patches.shape

(2, 2, 2, 3)

In [11]:
steps = [('pe', PatchExtractor()), ('SVM', SVC())]
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps) # define the pipeline object.

In [12]:
X = df_all.img_array


y = df_all.category_id

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test, X_validate, y_test, y_validate = train_test_split(X_test, y_test, test_size = 0.5)

In [13]:
parameters = {'SVM__C':[0.001,0.1,10,100,10e5], 'SVM__gamma':[0.1,0.01]}

In [15]:
grid = GridSearchCV(pipeline, param_grid=parameters, cv=5)

In [16]:
grid.fit(X_train, y_train)
print("score = %3.2f" %(grid.score(X_test,y_test)))
print(grid.best_params_)

ValueError: 
All the 50 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/home/siobhan/miniconda3/envs/fb-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/siobhan/miniconda3/envs/fb-env/lib/python3.10/site-packages/sklearn/pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/siobhan/miniconda3/envs/fb-env/lib/python3.10/site-packages/sklearn/pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/siobhan/miniconda3/envs/fb-env/lib/python3.10/site-packages/joblib/memory.py", line 349, in __call__
    return se

In [8]:
# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

In [10]:
# Learn the images on the train subset
clf.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [ ]:
# Predict the value of the digit on the test subset
predicted = clf.predict(X_test)

In [ ]:
import skimage.io as skio
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score


# Load the data
trainingImages = skio.imread_collection('./images_clean/*.jpg',conserve_memory=True)

# cast to numpy arrays
trainingImages = np.asarray(trainingImages)

# reshape img array to vector
def reshape_image(img):
    return np.reshape(img,len(img)*len(img[0]))

img_reshape = np.zeros((len(trainingImages),len(trainingImages[0])*len(trainingImages[0][0])))

for i in range(0,len(trainingImages)):
    img_reshape[i] = reshape_image(trainingImages[i])

# SVM
clf = svm.SVC(gamma=0.01,C=10,kernel='poly')
clf.fit(img_reshape, trainingTargets.values.tolist())